<a href="https://colab.research.google.com/github/GiharaNavindu/hugging-face-llm/blob/main/OCR-01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


In [2]:
# Install the AI model libraries and the tunneling tool
!pip install -q git+https://github.com/huggingface/transformers
!pip install -q accelerate bitsandbytes qwen-vl-utils streamlit
!npm install localtunnel

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
⠙⠹⠸⠼
up to date, audited 23 packages in 655ms
⠼
⠼3 packages are looking for funding
⠼  run `npm fund` for details
⠼
2 high severity vulnerabilities

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.
⠼

In [ ]:
%%writefile app.py
import streamlit as st
import torch
from transformers import Qwen2_5_VLForConditionalGeneration, AutoProcessor
from qwen_vl_utils import process_vision_info
from PIL import Image, ImageDraw
import json
import os

# --- APP SETUP ---
st.set_page_config(page_title="OCR", layout="wide")
st.title("🚀 Qwen 2.5 OCR")

# --- SIDEBAR ---
uploaded_file = st.sidebar.file_uploader("Upload Image", type=["jpg", "png"])
temp_path = "temp.jpg"

if uploaded_file:
    image = Image.open(uploaded_file).convert("RGB")
    st.sidebar.image(image, caption="Uploaded", use_column_width=True)
    image.save(temp_path)

# --- MODEL LOADER ---
@st.cache_resource
def load_model():
    from transformers import BitsAndBytesConfig
    # 4-bit quantization for free Colab T4 GPU
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.bfloat16
    )
    model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
        "Qwen/Qwen2.5-VL-7B-Instruct",
        quantization_config=bnb_config,
        device_map="auto"
    )
    processor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-7B-Instruct")
    return model, processor

# Trigger load
if uploaded_file:
    with st.spinner("Waking up Remote GPU..."):
        model, processor = load_model()

    if st.button("🔍 Extract Text"):
        messages = [{
            "role": "user", 
            "content": [
                {"type": "image", "image": temp_path},
                {"type": "text", "text": "Extract all text from this image."}
            ]
        }]
        
        # Prepare inputs
        text_input = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
        image_inputs, _ = process_vision_info(messages)
        inputs = processor(
            text=[text_input], 
            images=image_inputs, 
            padding=True, 
            return_tensors="pt"
        ).to(model.device)
        
        # Generate
        generated_ids = model.generate(**inputs, max_new_tokens=1024)
        output_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
        result = output_text.split("assistant\n")[-1]
        
        st.write("### Result:")
        st.write(result)

Writing app.py


In [ ]:
import urllib
# 1. Get the password for the tunnel
print("COPY THIS PASSWORD:", urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))

# 2. Run Streamlit in the background
!streamlit run app.py &>/content/logs.txt &
print("Streamlit app is running...")

# 3. Create the public tunnel
!npx localtunnel --port 8501
print("Tunnel created. Access your app via the provided URL.")

COPY THIS PASSWORD: 35.203.189.84
Streamlit app is running...
⠙your url is: https://great-pumas-win.loca.lt
